# Word2Vec 모델을 통해 증상별 질병 찾기

# 관련 모듈 설치

In [2]:
!pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.8 MB 871.5 kB/s eta 0:00:13
   -- ------------------------------------- 0.6/10.8 MB 7.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/10.8 MB 8.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.8 MB 9.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.8 MB 9.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.7/10.8 MB 9.7 MB/s eta 0:00:01
   ------------ --------------------------- 3.3/10.8 MB 10.0 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.8 MB 10.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.4/10.8 MB 10.3 MB/s eta 0:00:01
   ------------------ --------------------- 4.9/10.8 MB 10.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.6/10.8 MB 10.9 MB/s eta 0:00:01
   -----

In [3]:
!pip install numpy

In [4]:
pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
   ---------------------------------------- 0.0/351.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/351.7 kB ? eta -:--:--
   ------ --------------------------------- 61.4/351.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 351.7/351.7 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------ --------------------------------- 0.6/3.8 MB 17.6 MB/s eta 0:00:01
   ----------- ---------------------------- 1.1/3.8 MB 12.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.9/3.8 MB 11.8 MB/s eta 0:00:01
   ------------------------- -------------- 2.4/3.8 MB 11.7 MB/s eta 0:00:01
   ------------------------------ --------- 2.9/3.8 MB 11.8 MB/s eta 0:00:01
   ------------------------------------ --- 3.5/3.8 MB 11.7 MB/s eta 0:00:01
   --------------------------------------- 

In [5]:
pip install gensim

     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.0 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 59.0/59.0 kB 1.0 MB/s eta 0:00:00
  Using cached smart_open-7.0.4-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
    --------------------------------------- 0.5/24.0 MB 10.9 MB/s eta 0:00:03
   -- ------------------------------------- 1.2/24.0 MB 13.1 MB/s eta 0:00:02
   --- ------------------------------------ 1.9/24.0 MB 12.4 MB/s eta 0:00:02
   ---- ----------------------------------- 2.4/24.0 MB 12.0 MB/s eta 0:00:02
   ---- ----------------------------------- 3.0/24.0 MB 11.8 MB/s eta 0:00:02
   ----- ---------------------------------- 3.5/24.0 MB 11.7 MB/s eta 0:00:02
   ------ --------------------------------- 4.0/24.0 MB 11.7 MB/s eta 0:00:02
   ------- -------------------------------- 4.6/24.0 MB 11.7 MB/s eta 0:00:02
   -----

- 오류
- 파이썬 3.12버전에서는 gensim 설치시 오류 발생
- 3.8버전으로 환경 설정

# 관련 모듈 임포트

In [2]:
import pandas as pd
from gensim.models import Word2Vec
from konlpy.tag import Okt  # 한국어 처리를 위한 konlpy 라이브러리
import numpy as np

# 모델 학습 및 결과

In [4]:
import numpy as np
from gensim.models import Word2Vec

# CSV 파일 경로
file_path = './../../Data/disease_symptom_final.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)
df = df.dropna(subset=['symptoms'])

# 결과를 담을 리스트 초기화
result_lists = []

# 각 행(질병)에 대해 반복
for index, row in df.iterrows():
    disease_name = row['disease_name']
    symptoms = row['symptoms'].split(', ')
    
    # 첫 번째 형식의 리스트 생성  
    list1 = []
    for symptom in symptoms:
        list1.extend([disease_name, symptom])
    
    # 두 번째 형식의 리스트 생성
    list2 = [disease_name] + symptoms
    
    # 결과 리스트에 추가
    result_lists.append(list1)
    result_lists.append(list2)


# 질병 리스트와 증상 리스트 초기화
disease_list = []
symptom_list = []

# 각 행(질병)에 대해 반복
for index, row in df.iterrows():
    # 질병명과 질병 ID 추가
    disease_id = row['disease_id']
    disease_name = row['disease_name']
    if (disease_id, disease_name) not in disease_list:
        disease_list.append((disease_id, disease_name))
    
    # 증상 추가
    symptoms = row['symptoms'].split(', ')
    for symptom in symptoms:
        if symptom not in symptom_list:
            symptom_list.append(symptom)

# 예시 데이터와 질병 목록을 가정
sentences = result_lists  # 예시 데이터
diseases = [disease_name for disease_id, disease_name in disease_list]

# 모델 학습
model = Word2Vec(sentences, vector_size=150, window=5, min_count=1, workers=4, sg=1)

input_symptoms = ['분출성 구토']

# 각 증상에 대해 가장 유사한 질병과 그 유사도 찾기
symptom_to_diseases_similarity = {}
for symptom in input_symptoms:
    similar_words = model.wv.most_similar(symptom, topn=300)
    symptom_to_diseases_similarity[symptom] = [(word, similarity) for word, similarity in similar_words if word in diseases]

# 질병별로 증상과의 유사도 및 일치하는 증상 수 저장
disease_similarity_info = {}
for symptom, diseases_similarities in symptom_to_diseases_similarity.items():
    for disease, similarity in diseases_similarities:
        if disease in disease_similarity_info:
            disease_similarity_info[disease]['count'] += 1
            disease_similarity_info[disease]['similarities'].append(similarity)
        else:
            disease_similarity_info[disease] = {'count': 1, 'similarities': [similarity]}

# 증상이 2개 이상 일치하는 질병 필터링 및 평균 유사도 계산
filtered_diseases = [
    (disease, info['count'], np.mean(info['similarities']), info['similarities']) 
    for disease, info in disease_similarity_info.items() 
    if info['count'] >= 2 or len(info['similarities']) == 1
    ]

# 증상 일치 수 및 평균 유사도에 따라 정렬
filtered_diseases.sort(key=lambda x: (x[1], x[2]), reverse=True)

# 상위 10개 출력
for disease, count, avg_similarity, similarities in filtered_diseases[:10]:
    # 질병 ID를 가져오기
    disease_id = next(disease_id for disease_id, disease_name in disease_list if disease_name == disease)
    print(f"disease_id: {disease_id}, disease: {disease}, count: {count}, avg_similarity: {avg_similarity:.4f}, similarities: {similarities}")
    # count: 일치하는 증상 수: , avg_similarity:평균 유사도, similarities:증상별 유사도


disease_id: 32179, disease: 협심증(Angina pectoris), count: 1, avg_similarity: 0.9934, similarities: [0.9934274554252625]
disease_id: 32308, disease: 저나트륨혈증(Hyponatremia), count: 1, avg_similarity: 0.9933, similarities: [0.9932957291603088]
disease_id: 32051, disease: 위 정맥류(Gastric varices), count: 1, avg_similarity: 0.9932, similarities: [0.9932347536087036]
disease_id: 31836, disease: 아두골반 불균형(Cephalopelvic disproportion), count: 1, avg_similarity: 0.9932, similarities: [0.9932096600532532]
disease_id: 32386, disease: 중쇄 아실코에이 탈수소효소 결핍증(Medium-Chain Acyl-Coenzyme A Dehydrogenase Deficiency(MCAD deficiency)), count: 1, avg_similarity: 0.9931, similarities: [0.9931275248527527]
disease_id: 32209, disease: 요로결석(Urolithiasis), count: 1, avg_similarity: 0.9931, similarities: [0.9930999875068665]
disease_id: 31776, disease: 대장의 양성 종양(Benign colon tumor), count: 1, avg_similarity: 0.9931, similarities: [0.9930605292320251]
disease_id: 32100, disease: 헬리코박터균 감염(Helicobacter pylori infection), c

# 모델, 데이터 저장

In [6]:
import pickle


with open('disease_list.pkl', 'wb') as f:
    pickle.dump(disease_list, f)

In [7]:
model.save("model/240517_final.model")